# DAG Dependencies

Often, parallel workflow is described in terms of a [Directed Acyclic
Graph](http://en.wikipedia.org/wiki/Directed_acyclic_graph) or DAG. A
popular library for working with Graphs is
[NetworkX](http://networkx.lanl.gov/). Here, we will walk through a demo
mapping a nx DAG to task dependencies.

The full script that runs this demo can be found in `code/dagdeps.py`.

## Why are DAGs good for task dependencies?

The 'G' in DAG is 'Graph'. A Graph is a collection of **nodes** and
**edges** that connect the nodes. For our purposes, each node would be a
task, and each edge would be a dependency. The 'D' in DAG stands for
'Directed'. This means that each edge has a direction associated with
it. So we can interpret the edge (a,b) as meaning that b depends on a,
whereas the edge (b,a) would mean a depends on b. The 'A' is 'Acyclic',
meaning that there must not be any closed loops in the graph. This is
important for dependencies, because if a loop were closed, then a task
could ultimately depend on itself, and never be able to run. If your
workflow can be described as a DAG, then it is impossible for your
dependencies to cause a deadlock.

## A Sample DAG

Here, we have a very simple 5-node DAG:

In [15]:
%matplotlib inline
import matplotlib.pyplot as plt

import ipyparallel as parallel

rc = parallel.Client()
view = rc.load_balanced_view()

In [3]:
ar = view.apply_async(time.sleep, 3)
with view.temp_flags(after=ar, timeout=2):
    ar2 = view.apply_async(time.sleep, 2)

%time ar2.wait()

In [4]:
ar2.get()

In [5]:
import networkx as nx

G = nx.DiGraph()

# add 5 nodes, labeled 0-4:
map(G.add_node, range(5))
# 1,2 depend on 0:
G.add_edge(0,1)
G.add_edge(0,2)
# 3 depends on 1,2
G.add_edge(1,3)
G.add_edge(2,3)
# 4 depends on 1
G.add_edge(1,4)

# now draw the graph:
pos = { 0 : (0,0), 1 : (1,1), 2 : (-1,1),
        3 : (0,2), 4 : (2,2)}
nx.draw(G, pos, edge_color='r')

With NetworkX, an arrow is just a fattened bit on the edge. Here, we can
see that task 0 depends on nothing, and can run immediately. 1 and 2
depend on 0; 3 depends on 1 and 2; and 4 depends only on 1.

A possible sequence of events for this workflow:

0.  Task 0 can run right away
1.  0 finishes, so 1,2 can start
2.  1 finishes, 3 is still waiting on 2, but 4 can start right away
3.  2 finishes, and 3 can finally start

Further, taking failures into account, assuming all dependencies are run
with the default `success=True,failure=False`, the following cases
would occur for each node's failure:

0.  fails: all other tasks fail as Impossible
1.  2 can still succeed, but 3,4 are unreachable
2.  3 becomes unreachable, but 4 is unaffected
3.  and 4. are terminal, and can have no effect on other nodes

For demonstration purposes, we have a function that generates a random DAG with a given number of nodes and edges.

In [6]:
from random import randint
def random_dag(nodes, edges):
    """Generate a random Directed Acyclic Graph (DAG) with a given number of nodes and edges."""
    G = nx.DiGraph()
    for i in range(nodes):
        G.add_node(i)
    while edges > 0:
        a = randint(0,nodes-1)
        b=a
        while b==a:
            b = randint(0,nodes-1)
        G.add_edge(a,b)
        if nx.is_directed_acyclic_graph(G):
            edges -= 1
        else:
            # we closed a loop!
            G.remove_edge(a,b)
    return G

So first, we start with a graph of 32 nodes, with 128 edges:

In [7]:
G = random_dag(32,128)

In [8]:
nx.draw(G, pos=nx.spring_layout(G))

In [9]:
def randomwait():
    import time, random
    tic = time.time()
    time.sleep(random.random())
    return time.time() - tic

Once we have a dict of jobs matching the nodes on the graph, 
we can start submitting jobs, and linking up the dependencies. 
Since we don’t know a job’s msg_id until it is submitted, 
which is necessary for building dependencies, 
it is critical that we don’t submit any jobs before other jobs it may depend on. 
Fortunately, NetworkX provides a topological_sort() method which ensures exactly this. 
It presents an iterable, that guarantees that when you arrive at a node, 
you have already visited all the nodes it on which it depends:

In [10]:
jobs = {}

# in reality, each job would presumably be different
# randomwait is just a function that sleeps for a random interval
for node in G:
     jobs[node] = randomwait

Once we have a dict of jobs matching the nodes on the graph, we can
start submitting jobs, and linking up the dependencies. Since we don't
know a job's msg_id until it is submitted, which is necessary for
building dependencies, it is critical that we don't submit any jobs
before other jobs it may depend on. Fortunately, NetworkX provides a
`topological_sort` method which ensures exactly this. It presents an
iterable, that guarantees that when you arrive at a node, you have
already visited all the nodes it on which it depends:

In [11]:
results = {}

for node in nx.topological_sort(G):
    # get list of AsyncResult objects from nodes
    # leading into this one as dependencies
    deps = [ results[n] for n in G.predecessors(node) ]
    # submit and store AsyncResult object
    with view.temp_flags(after=deps, block=False):
         results[node] = view.apply_async(jobs[node])

In [12]:
view.wait()

In [13]:
def validate_tree(G, results):
    """Validate that jobs executed after their dependencies."""
    for node in G:
        started = results[node].metadata.started
        for parent in G.predecessors(node):
            finished = results[parent].metadata.completed
            assert started > finished, "%s should have happened after %s" % (node, parent)

validate_tree(G, results)

In [16]:
from matplotlib.dates import date2num

from matplotlib.cm import gist_rainbow

pos = {}; colors = {}

xmin = ymin = np.inf
xmax = ymax = 0

for node in G:
    md = results[node].metadata
    start = date2num(md.started)
    if start < xmin:
        xmin = start
    if start > xmax:
        xmax = start
    runtime = date2num(md.completed) - start
    if runtime < ymin:
        ymin = runtime
    if runtime > ymax:
        ymax = runtime
    pos[node] = (start, runtime)
    colors[node] = md.engine_id

nx.draw(G, pos, node_list=colors.keys(), node_color=colors.values(),
    cmap=gist_rainbow)

dx = xmax - xmin
dy = ymax - ymin
plt.xlim(xmin-0.1*dx, xmax+0.1*dx)
plt.ylim(ymin-0.1*dy, ymax+0.1*dy)

Time started on x, runtime on y, and color-coded by engine-id (in this case there were four engines). Edges denote dependencies.